# MLOps workshop with Amazon SageMaker

## Module 03: Automate the whole dataset preparation and model training pipeline with SageMaker Pipelines.

In the last module of this workshop we will create a repeatable production workflow which is typically run outside notebooks. To demonstrate automating the workflow, we'll use [Amazon SageMaker Pipelines](https://aws.amazon.com/sagemaker/pipelines) for workflow orchestration. Purpose-built for machine learning (ML), SageMaker Pipelines helps you automate different steps of the ML workflow including data processing, model training, and batch prediction (scoring), and apply conditions such as approvals for model quality. It also includes a model registry and model lineage tracker.

## Workflow Automation with SageMaker Pipelines <a class="anchor" id="WorkflowAutomation">

In the previous parts of this notebook, we prototyped various steps of a TensorFlow project within the notebook itself, with some steps being run on external SageMaker resources (hosted training, model tuning, hosted endpoints).  Notebooks are great for prototyping, but generally are  not used in production-ready machine learning pipelines.  

A very simple pipeline in SageMaker includes processing the dataset to get it ready for training, performing the actual training, and then using the model to perform some form of inference such as batch predition (scoring). We'll use SageMaker Pipelines to automate these steps, keeping the pipeline simple for now: it easily can be extended into a far more complex pipeline.

### Pipeline parameters <a class="anchor" id="PipelineParameters">

Before we begin to create the pipeline itself, we should think about how to parameterize it.  For example, we may use different instance types for different purposes, such as CPU-based types for data processing and GPU-based or more powerful types for model training.  These are all "knobs" of the pipeline that we can parameterize.  Parameterizing enables custom pipeline executions and schedules without having to modify the pipeline definition.

In [ ]:
!pip install --upgrade sagemaker -q

In [ ]:
import os
import sagemaker

sess = sagemaker.session.Session()
bucket = sess.default_bucket() 

raw_s3 = f"s3://{bucket}/tf-2-workflow/data/raw"    # sess.upload_data(path='./data/raw/', key_prefix=rawdata_s3_prefix)

print(sagemaker.__version__)
print(raw_s3)

### Pipline parameters

We will create the pipeline parameters, that later we could use them to make the pipeline configurable without updating it.
For example, we will create `processing_instance_type` that will default to `ml.m5.xlarge` but during the pipeline invocation we will be able to modify it

In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

# raw input data
input_data = ParameterString(name="InputData", default_value=raw_s3)

# processing step parameters
processing_instance_type = ParameterString(name="ProcessingInstanceType", default_value="ml.m5.xlarge")
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)

# training step parameters
training_instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.c5.2xlarge")
training_instance_count = ParameterInteger(name="TrainingInstanceCount", default_value=1)

# batch inference step parameters
batch_instance_type = ParameterString(name="BatchInstanceType", default_value="ml.c5.xlarge")
batch_instance_count = ParameterInteger(name="BatchInstanceCount", default_value=1)

### Processing Step <a class="anchor" id="ProcessingStep">

The first step in the pipeline will preprocess the data to prepare it for training. We create a `SKLearnProcessor` object similar to the one above, but now parameterized so we can separately track and change the job configuration as needed, for example to increase the instance type size and count to accommodate a growing dataset.

In [ ]:
%%writefile preprocessing.py

import glob
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

if __name__=='__main__':
    
    input_files = glob.glob('{}/*.npy'.format('/opt/ml/processing/input'))
    print('\nINPUT FILE LIST: \n{}\n'.format(input_files))
    scaler = StandardScaler()
    x_train = np.load(os.path.join('/opt/ml/processing/input', 'x_train.npy'))
    scaler.fit(x_train)
    for file in input_files:
        raw = np.load(file)
        # only transform feature columns
        if 'y_' not in file:
            transformed = scaler.transform(raw)
        if 'train' in file:
            if 'y_' in file:
                output_path = os.path.join('/opt/ml/processing/train', 'y_train.npy')
                np.save(output_path, raw)
                print('SAVED LABEL TRAINING DATA FILE\n')
            else:
                output_path = os.path.join('/opt/ml/processing/train', 'x_train.npy')
                np.save(output_path, transformed)
                print('SAVED TRANSFORMED TRAINING DATA FILE\n')
        else:
            if 'y_' in file:
                output_path = os.path.join('/opt/ml/processing/test', 'y_test.npy')
                np.save(output_path, raw)
                print('SAVED LABEL TEST DATA FILE\n')
            else:
                output_path = os.path.join('/opt/ml/processing/test', 'x_test.npy')
                np.save(output_path, transformed)
                print('SAVED TRANSFORMED TEST DATA FILE\n')

In [ ]:
import sagemaker
import boto3


def get_sagemaker_execution_role():
    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20210510T183593')['Role']['Arn'] # When updating locally update your sagemaker execution code, to get it working locally
    
    return role

In [ ]:
import boto3
import sagemaker
from sagemaker.sklearn.processing import SKLearnProcessor

sess = sagemaker.session.Session()
bucket = sess.default_bucket() 
role = get_sagemaker_execution_role()
print(role)
region = boto3.Session().region_name
framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="tf-2-workflow-process",
    sagemaker_session=sess,
    role=role,
)

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep


step_process = ProcessingStep(
    name="TF2Process",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=input_data, destination="/opt/ml/processing/input", s3_data_distribution_type='ShardedByS3Key'),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    code="./preprocessing.py",
)

### Training and Model Creation Steps <a class="anchor" id="TrainingModelCreation">

The following code sets up a pipeline step for a training job. We start by specifying which SageMaker prebuilt TensorFlow 2 training container to use for the job.

In [ ]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.step_collections import RegisterModel

tensorflow_version = '2.6'
python_version = 'py38'

image_uri_train = sagemaker.image_uris.retrieve(
                                        framework="tensorflow",
                                        region=region,
                                        version=tensorflow_version,
                                        py_version=python_version,
                                        instance_type=training_instance_type,
                                        image_scope="training"
                                       )

Next, we specify an `Estimator` object, and define a `TrainingStep` to insert the training job in the pipeline with inputs from the previous SageMaker Processing step. Notice that we have used the hyperparameters from the best estimator in the tuning job we ran before. AutoTuning integration with SageMaker Pipelines is now available, but not yet integrated into this notebook (will be updated soon). We should use it to find the best model.

In [ ]:
import time

model_path = f"s3://{bucket}/TF2WorkflowTrain"
training_parameters = {'epochs': 20, 'batch_size': 128, 'learning_rate': 0.01, 'for_pipeline': 'true'}

estimator = TensorFlow(
    image_uri=image_uri_train,
    source_dir='code',
    entry_point='train.py',
    instance_type=training_instance_type,
    instance_count=training_instance_count,
    role=role,
    base_job_name="tf-2-workflow-train",
    output_path=model_path,
    hyperparameters=training_parameters
)

In [ ]:
step_train = TrainingStep(
    name="TF2WorkflowTrain",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri
        ),
        "test": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri
        )
    },
)

As another step, we create a SageMaker `Model` object to wrap the model artifact, and associate it with a separate SageMaker prebuilt TensorFlow Serving inference container to potentially use later.

In [ ]:
from sagemaker.model import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep

image_uri_inference = sagemaker.image_uris.retrieve(
                                        framework="tensorflow",
                                        region=region,
                                        version=tensorflow_version,
                                        py_version=python_version,
                                        instance_type=batch_instance_type,
                                        image_scope="inference"
                                       )
model = Model(
    image_uri=image_uri_inference,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sess,
    role=role,
)

inputs_model = CreateModelInput(
    instance_type=batch_instance_type
)

step_create_model = CreateModelStep(
    name="TF2WorkflowCreateModel",
    model=model,
    inputs=inputs_model,
)

### Batch Scoring Step <a class="anchor" id="BatchScoringStep">
    
The final step in this pipeline is offline, batch scoring (inference/prediction).  The inputs to this step will be the model we trained earlier, and the test data.  A simple, ordinary Python script is all we need to do the actual batch inference.

In [ ]:
%%writefile batch-score.py

import os
import subprocess
import sys
import numpy as np
import pathlib
import tarfile


if __name__ == "__main__":
    
    model_path = f"/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path, 'r:gz') as tar:
        tar.extractall('./model')
    import tensorflow as tf
    model = tf.keras.models.load_model('./model/1')
    test_path = "/opt/ml/processing/test/"
    x_test = np.load(os.path.join(test_path, 'x_test.npy'))
    y_test = np.load(os.path.join(test_path, 'y_test.npy'))
    scores = model.evaluate(x_test, y_test, verbose=2)
    print("\nTest MSE :", scores)
    
    output_dir = "/opt/ml/processing/batch"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)
    evaluation_path = f"{output_dir}/score-report.txt"
    with open(evaluation_path, 'w') as writer:
         writer.write(f"Test MSE : {scores}")

We'll use SageMaker Processing here to perform batch scoring.

In [ ]:
from sagemaker.processing import FrameworkProcessor
from sagemaker.tensorflow import TensorFlow


step_batch_name = "TF2WorkflowBatchScoring"

batch_scorer = FrameworkProcessor(
    estimator_cls=TensorFlow,
    framework_version=tensorflow_version,
    role=role,
    instance_count=batch_instance_count,
    instance_type=batch_instance_type,
    py_version='py38',
    command=["python3"],
    base_job_name="tf-2-workflow-batch"
)

batch_scorer_run_args = batch_scorer.get_run_args(
    code="batch-score.py",
    source_dir="./",
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="batch", source="/opt/ml/processing/batch"),
    ],
)

step_batch = ProcessingStep(
    name=step_batch_name,
    processor=batch_scorer,
    inputs=batch_scorer_run_args.inputs,
    outputs=batch_scorer_run_args.outputs,
    code=batch_scorer_run_args.code
)

### Creating and executing the pipeline <a class="anchor" id="CreatingExecutingPipeline">

With all of the pipeline steps now defined, we can define the pipeline itself as a `Pipeline` object comprising a series of those steps.  Parallel and conditional steps also are possible.

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"TF2Workflow"

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[input_data,
                processing_instance_type, 
                processing_instance_count, 
                training_instance_type, 
                training_instance_count,
                batch_instance_type,
                batch_instance_count],
    steps=[step_process, 
           step_train, 
           step_create_model,
           step_batch
          ],
    sagemaker_session=sess
)

We can inspect the pipeline definition in JSON format:

In [ ]:
import json

definition = json.loads(pipeline.definition())

# Show the pipeline definition json
definition

After upserting its definition, we can start the pipeline with the `Pipeline` object's `start` method:

In [ ]:
# upsert pipeline, unique by name
pipeline.upsert(role_arn=role)

# start the pipeline
execution = pipeline.start()

We can now confirm that the pipeline is executing.  In the log output below, confirm that `PipelineExecutionStatus` is `Executing`.

We wait for 30 seconds for the pipeline to start executing in order to get the pipeline metadata 

In [ ]:
time.sleep(30)

In [ ]:
execution.describe()

In [ ]:
trial_name = execution.describe()['PipelineExperimentConfig']['TrialName']
trial_name

#### Review the Pipeline

After the pipeline started executing, you can view the pipeline run. 

To view them, open SageMaker studio click the home button, click Pipelines, and locate your pipeline TF2Workflow.

![sageMakers_components_and_registries_button](./img/sagemaker-pipeline-1.jpg)


Click the `TF2Workflow` pipeline, and then double click on the execution.

![click_the_pipeline_execution](./img/pipeline-execution.jpg)


Now you can see the pipeline executing. Click on `TF2Process` step to see additional details.

![view_pipeline_execution](./img/pipeline-execution-graph.jpg)


On this specific step, you'll be able to see the output, logs and information.

![view_step_details](./img/pipeline-training-logs.jpg)

Typically this pipeline should take about 10 minutes to complete.  We can wait for completion by invoking `wait()`. After execution is complete, we can list the status of the pipeline steps.

In [ ]:
execution.wait()
execution.list_steps()

### Check the score report

After the batch scoring job in the pipeline is complete, the batch scoring report is uploaded to S3.  For simplicity, this report simply states the test MSE, but in general reports can include as much detail as desired.  Reports such as these also can be formatted for use in conditional approval steps in SageMaker Pipelines.  For example, the pipeline could have a condition step that only allows further steps to proceed only if the MSE is lower than some threshold.  

In [ ]:
report_path = f"s3://{bucket}/{pipeline_name}/{trial_name}/{step_batch_name}/output/batch/score-report.txt"
!aws s3 cp {report_path} ./score-report.txt --quiet && cat score-report.txt

## ML Lineage Tracking <a class="anchor" id="LineageOfPipelineArtifacts">

SageMaker ML Lineage Tracking creates and stores information about the steps of a ML workflow from data preparation to model deployment. With the tracking information you can reproduce the workflow steps, track model and dataset lineage, and establish model governance and audit standards.

Let's now check out the lineage of the model generated by the pipeline above.  The lineage table identifies the resources used in training, including the timestamped train and test data sources, and the specific version of the TensorFlow 2 container in use during the training job.  

In [ ]:
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(execution.list_steps()):
    if execution_step['StepName'] == 'TF2WorkflowTrain':
        display(viz.show(pipeline_execution_step=execution_step))

## Extensions <a class="anchor" id="Extensions">

We've covered a lot of content in this notebook:  SageMaker Processing for data transformation, Automatic Model Tuning, and SageMaker hosted training and inference.  These are central elements for most deep learning workflows in SageMaker.  Additionally, we examined how SageMaker Pipelines helps automate deep learning workflows after completion of the prototyping phase of a project.

Besides all of the SageMaker features explored above, there are many other features that may be applicable to your project.  For example, to handle common problems during deep learning model training such as vanishing or exploding gradients, **SageMaker Debugger** is useful.  To manage common problems such as data drift after a model is in production, **SageMaker Model Monitor** can be applied.